In [8]:
import os

from surprise import SVD
from surprise import Dataset
from surprise import accuracy
from surprise.model_selection import train_test_split
from surprise import Reader

# Load the data set from file
file_path = os.path.join("crawl-ratings", "train.csv")
reader = Reader(
    line_format="user item rating", sep=",", rating_scale=(1, 10), skip_lines=1
)
data = Dataset.load_from_file(file_path, reader=reader)

# sample random trainset and validation set
# val set is made of 25% of the ratings.
train_set, validation_set = train_test_split(data, test_size=0.25)

# We'll use the famous SVD algorithm.
algo = SVD()

# Train the algorithm on the trainset, and predict ratings for the testset
algo.fit(train_set)
predictions = algo.test(test_set)

# Then compute RMSE
accuracy.rmse(predictions)

RMSE: 1.5714


1.571386647224214